# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,11.16,91,100,0.89,US,1709753059
1,1,bethel,41.3712,-73.4140,11.47,85,75,1.54,US,1709753059
2,2,grytviken,-54.2811,-36.5092,7.11,56,100,1.12,GS,1709753059
3,3,waitangi,-43.9535,-176.5597,12.12,83,98,4.05,NZ,1709753060
4,4,port saint john's,-31.6229,29.5448,21.84,72,38,2.25,ZA,1709753060


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,tecalitlan,19.4667,-103.3000,24.25,18,0,2.72,MX,1709753074
103,103,monywa,22.1167,95.1333,23.68,30,0,1.58,MM,1709753096
263,263,howrah,22.5892,88.3103,24.00,64,0,0.00,IN,1709753155
271,271,al hazm,16.1641,44.7769,26.06,41,0,2.07,YE,1709753158
296,296,al ghayzah,16.2079,52.1760,26.11,76,0,4.14,YE,1709753168
351,351,piti village,13.4626,144.6933,26.03,87,0,2.06,GU,1709753188
352,352,santa maria,-29.6842,-53.8069,26.84,38,0,2.98,BR,1709752783
395,395,agat village,13.3886,144.6585,26.06,88,0,2.06,GU,1709753204
536,536,raghunathpur,23.5500,86.6667,22.22,17,0,2.97,IN,1709753257
544,544,coscomatepec de bravo,19.0667,-97.0333,26.41,44,0,3.49,MX,1709753260


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_dict={
    'City': city,
    'Country': country,
    'Lat': lat,
    'Lng': lng,
}
hotel_df=pd.DataFrame(hotel_dict)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Hotel Name
42,tecalitlan,MX,19.4667,-103.3000,
103,monywa,MM,22.1167,95.1333,
263,howrah,IN,22.5892,88.3103,
271,al hazm,YE,16.1641,44.7769,
296,al ghayzah,YE,16.2079,52.1760,
351,piti village,GU,13.4626,144.6933,
352,santa maria,BR,-29.6842,-53.8069,
395,agat village,GU,13.3886,144.6585,
536,raghunathpur,IN,23.5500,86.6667,
544,coscomatepec de bravo,MX,19.0667,-97.0333,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tecalitlan - nearest hotel: Hotel San Francisco
monywa - nearest hotel: Hotel Chindwin
howrah - nearest hotel: Sun India Guest House
al hazm - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
piti village - nearest hotel: The Cliff Hotel
santa maria - nearest hotel: Hotel Morotin
agat village - nearest hotel: Navy Getway Inns & Suites
raghunathpur - nearest hotel: No hotel found
coscomatepec de bravo - nearest hotel: Ecos del Viento


,City,Country,Lat,Lng,Hotel Name
42,tecalitlan,MX,19.4667,-103.3000,Hotel San Francisco
103,monywa,MM,22.1167,95.1333,Hotel Chindwin
263,howrah,IN,22.5892,88.3103,Sun India Guest House
271,al hazm,YE,16.1641,44.7769,No hotel found
296,al ghayzah,YE,16.2079,52.1760,فندق تاج العرب
351,piti village,GU,13.4626,144.6933,The Cliff Hotel
352,santa maria,BR,-29.6842,-53.8069,Hotel Morotin
395,agat village,GU,13.3886,144.6585,Navy Getway Inns & Suites
536,raghunathpur,IN,23.5500,86.6667,No hotel found
544,coscomatepec de bravo,MX,19.0667,-97.0333,Ecos del Viento


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"])

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)